# Convergence workflow
## Kat Nykiel

This notebook provides a demonstration of how to determine convergence for a given DTM MXene system

### Define helper functions

Here, we define two functions we can use in nanoHUB to create and run simulations

In [ ]:
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure

def make_sim(name,struct,**kwargs):
    """
    Generate quantum espresso input files using pymatgen's PWInput class
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    **kwargs:
        dictionaries to input to pymatgen's PWInput object
    """
    # Prepare dict of pseudopotentials (i.e. {'Mg': 'Mg.upf', 'O': 'O.upf'})
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_dict = dict(zip(elements,[f"{element}.upf" for element in elements]))

    # Define input set
    input_set = PWInput(structure=struct,
                        pseudo=pseudo_dict,
                        **kwargs) # dictionaries corresponding to blocks in QE input files

    input_set.write_file(filename=f'{name}.in')
    
def run_sim(name,struct):
    """
    Submit QuantumESPRESSO runs to HPC clusters from nanoHUB
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    """
    # Write input and output files
    input_file = open(f'{name}.in','a')
    input_file.close()

    output_file = open(f'{name}.out', 'w')
    output_file.close()
    
    # Set up commands and files
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_arg = "".join([f"-i ./pseudo/pseudo_PAW/{element}.upf " for element in elements])
    COMMAND = f"espresso-6.8_pw > {output_file.name}"
    
    # Run simulation (1 node, 1 hour walltime)
    !submit -n 1 -w '01:00:00' -e QE_DISABLE_GGA_PBE=0 --runName {name} {COMMAND} {pseudo_arg} -i {input_file.name}

### Run a sample workflow

In [ ]:
# Load a DTM MXene structure
struct = Structure.from_file("DTM-MXene.vasp")

In [ ]:
# Create an ionic relaxation sim
make_sim("relax", struct,
         control={'pseudo_dir':'./',
                  'calculation':'relax',
                  'outdir':'./',
                  'tstress':True},
         system={'ecutwfc':50})

# Run relax simulation
run_sim("relax", struct)